In [2]:
import numpy as np
import pandas as pd
import scipy.stats

In [10]:
df = pd.read_csv("../dataset/results.csv")
df["filename"] = df["filename"].str.replace("resnet_bias", "resnet-bias").str.replace(".keras", "")
df[["dataset", "model", "loss", "depth"]] = df["filename"].str.split("_", expand=True, n=4)
df.head()

,filename,inference_time,mcc,accuracy,precision,recall,f1,roc_auc,ap,dataset,model,loss,depth
0,all-undersampled_resnet-bias_fl_6,8.579865,0.538098,0.971849,0.299621,0.995022,0.460559,0.983294,0.298190,all-undersampled,resnet-bias,fl,6
1,sel-10_resnet-bias_bce_6,6.085120,0.814527,0.995898,0.911376,0.731465,0.811569,0.865297,0.669882,sel-10,resnet-bias,bce,6
2,all_resnet-bias_bce_1,3.993372,0.936704,0.998510,0.952162,0.922976,0.937342,0.961205,0.879753,all,resnet-bias,bce,1
3,all-undersampled_resnet_bce_1,3.695442,0.449419,0.955650,0.213151,0.992795,0.350953,0.973996,0.211702,all-undersampled,resnet,bce,1
4,all_resnet_bce_6,5.494683,0.921962,0.998179,0.948899,0.897564,0.922518,0.948486,0.852934,all,resnet,bce,6


In [14]:
k1 = df[(df["dataset"] == "all") & (df["model"] == "fcn")]["accuracy"]
k2 = df[(df["dataset"] == "all") & (df["model"] == "resnet-bias")]["accuracy"]

In [15]:
res = scipy.stats.ttest_ind(k1, k2)
res

TtestResult(statistic=-1.8243343716114802, pvalue=0.08171418679972874, df=22.0)

In [16]:
res.confidence_interval()

ConfidenceInterval(low=-0.0007685026913842779, high=4.9194703637724276e-05)

In [21]:
scipy.stats.ttest_1samp(k2, 0.9)

TtestResult(statistic=1563.2737633293827, pvalue=9.218309119625006e-31, df=11)

## ANOVA

In [24]:
df_anova = df[["mcc", "dataset", "model", "loss", "depth"]]
df_anova

,mcc,dataset,model,loss,depth
0,0.538098,all-undersampled,resnet-bias,fl,6
1,0.814527,sel-10,resnet-bias,bce,6
2,0.936704,all,resnet-bias,bce,1
3,0.449419,all-undersampled,resnet,bce,1
4,0.921962,all,resnet,bce,6
...,...,...,...,...,...
139,0.940704,all,resnet,fl,2
140,0.479917,sel-10-undersampled,fcn,fl,2
141,0.506483,all-undersampled,resnet-bias,bce,3
142,0.520468,all-undersampled,fcn,fl,1


In [23]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [53]:
lm = ols("mcc ~ C(model) * C(loss) * C(depth)", data=df_anova).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mcc   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                 -0.265
Method:                 Least Squares   F-statistic:                    0.1438
Date:                Wed, 02 Oct 2024   Prob (F-statistic):               1.00
Time:                        10:49:24   Log-Likelihood:                 13.326
No. Observations:                 144   AIC:                             45.35
Df Residuals:                     108   BIC:                             152.3
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Intercept                                               0.5833      0.127      4.580      0.000       0.331       0.836
C(model)[T.resnet]                                      0.0580      0.180      0.322      0.748      -0.299       0.415
C(model)[T.resnet-bias]                                 0.1099      0.180      0.610      0.543      -0.247       0.467
C(loss)[T.fl]                                           0.0064      0.180      0.035      0.972      -0.351       0.363
C(depth)[T.2]                                           0.1019      0.180      0.566      0.573      -0.255       0.459
C(depth)[T.3]                                           0.1306      0.180      0.725      0.470      -0.226       0.488
C(depth)[T.4]                                           0.1409      0.180      0.783      0.436      -0.216       0.498
C(depth)[T.5]                                           0.1276      0.180      0.709      0.480      -0.229       0.485
C(depth)[T.6]                                           0.1433      0.180      0.796      0.428      -0.214       0.500
C(model)[T.resnet]:C(loss)[T.fl]                       -0.0653      0.255     -0.256      0.798      -0.570       0.440
C(model)[T.resnet-bias]:C(loss)[T.fl]                  -0.0028      0.255     -0.011      0.991      -0.508       0.502
C(model)[T.resnet]:C(depth)[T.2]                       -0.0976      0.255     -0.383      0.702      -0.602       0.407
C(model)[T.resnet-bias]:C(depth)[T.2]                  -0.0996      0.255     -0.391      0.697      -0.604       0.405
C(model)[T.resnet]:C(depth)[T.3]                       -0.1643      0.255     -0.645      0.520      -0.669       0.341
C(model)[T.resnet-bias]:C(depth)[T.3]                  -0.1372      0.255     -0.539      0.591      -0.642       0.368
C(model)[T.resnet]:C(depth)[T.4]                       -0.1579      0.255     -0.620      0.537      -0.663       0.347
C(model)[T.resnet-bias]:C(depth)[T.4]                  -0.1349      0.255     -0.530      0.598      -0.640       0.370
C(model)[T.resnet]:C(depth)[T.5]                       -0.1429      0.255     -0.561      0.576      -0.648       0.362
C(model)[T.resnet-bias]:C(depth)[T.5]                  -0.1429      0.255     -0.561      0.576      -0.648       0.362
C(model)[T.resnet]:C(depth)[T.6]                       -0.1970      0.255     -0.774      0.441      -0.702       0.308
C(model)[T.resnet-bias]:C(depth)[T.6]                  -0.1493      0.255     -0.586      0.559      -0.654       0.356
C(loss)[T.fl]:C(depth)[T.2]                             0.0024      0.255      0.010      0.992      -0.502       0.507
C(loss)[T.fl]:C(depth)[T.3]                             0.0059      0.255      0.02

In [54]:
anova_res = sm.stats.anova_lm(lm, typ=2)
anova_res

,sum_sq,df,F,PR(>F)
C(model),0.195074,2.0,1.503443,0.226980
C(loss),0.000215,1.0,0.003315,0.954190
C(depth),0.038794,5.0,0.119594,0.987794
C(model):C(loss),0.003858,2.0,0.029734,0.970712
C(model):C(depth),0.077373,10.0,0.119264,0.999560
C(loss):C(depth),0.002177,5.0,0.006711,0.999989
C(model):C(loss):C(depth),0.009102,10.0,0.014030,1.000000
Residual,7.006586,108.0,NaN,NaN
